In [10]:
import re


In [11]:
# from collections import Counter

# def find_recurring_phrases(text, min_length=20, threshold=2):
#     """
#     Identifies recurring phrases in the text based on frequency.

#     :param text: str, the input text.
#     :param min_length: int, minimum character length of a phrase to consider.
#     :param threshold: int, minimum occurrences to qualify as recurring.
#     :return: list of str, recurring phrases.
#     """
#     # Split into potential phrases (e.g., sentences or lines)
#     phrases = re.split(r'[.\n]', text)
#     phrases = [phrase.strip() for phrase in phrases if len(phrase.strip()) >= min_length]

#     # Count frequencies
#     phrase_counts = Counter(phrases)
#     recurring_phrases = [phrase for phrase, count in phrase_counts.items() if count >= threshold]

#     return recurring_phrases

# def remove_recurring_phrases_dynamically(text, min_length=20, threshold=2):
#     """
#     Finds and removes recurring phrases from the text dynamically.

#     :param text: str, the input text to clean.
#     :param min_length: int, minimum character length of a phrase to consider.
#     :param threshold: int, minimum occurrences to qualify as recurring.
#     :return: str, cleaned text.
#     """
#     # Identify recurring phrases
#     recurring_phrases = find_recurring_phrases(text, min_length, threshold)
    
#     # Remove recurring phrases
#     for phrase in recurring_phrases:
#         escaped_phrase = re.escape(phrase)  # Escape for regex
#         text = re.sub(escaped_phrase, '', text)

#     # Clean up extra spaces and newlines
#     text = re.sub(r'\s{2,}', ' ', text)
#     text = re.sub(r'\n{2,}', '\n', text)
    
#     return text, recurring_phrases

In [12]:
BASE_PATH = "../data/crawled_content"
UNIVERSITY_STR = ["uiw_edu", "stmary_edu", "klesse_utsa_edu"][-1]
full_path = f"{BASE_PATH}/{UNIVERSITY_STR}.txt"

print(f"Attempting to open: {full_path}")

with open(full_path, 'r', encoding='utf-8', errors='ignore') as file:
    content = file.read()    
    # print(content)

content = content.replace("|", "-")

Attempting to open: ../data/crawled_content/klesse_utsa_edu.txt


In [16]:

def robust_chunk_text_with_warning(text, max_chunk_size=2000):
    """
    Splits text into chunks, ensuring sections defined by LINK, TITLE, and CONTENT
    remain intact and are not split between chunks. Raises a message if any page exceeds max_chunk_size.
    
    :param text: str, the input text to split into chunks.
    :param max_chunk_size: int, the maximum size of each chunk.
    :return: list of str, the text chunks.
    """
    def preprocess_text(text):
        """Normalizes spacing and line breaks."""
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n+', '\n', text)
        return text

    # Preprocess text to normalize formatting
    text = preprocess_text(text)
    
    # Enhanced regex to allow for flexible spacing and line breaks
    pattern = r'(LINK: .*?TITLE: .*?CONTENT: .*?)(?=LINK:|$)'
    matches = re.findall(pattern, text, re.DOTALL)
    
    if not matches:
        print("No matches found. Check formatting or regex.")
        return []
    else:
        print(f"Found {len(matches)} matches.")
    
    # Check if any single page exceeds max_chunk_size
    for i, section in enumerate(matches):
        if len(section) > max_chunk_size:
            print(f"Warning: Page {i+1} exceeds max_chunk_size ({len(section)} characters). Please review:")
            print(section[:500] + "...\n")  # Show the first 500 characters for context
    
    chunks = []
    current_chunk = []
    current_size = 0

    for section in matches:
        section_size = len(section)
        
        # Check if adding this section exceeds the max chunk size
        if current_size + section_size > max_chunk_size and current_chunk:
            # Finalize the current chunk
            chunks.append(''.join(current_chunk).strip())
            current_chunk = []
            current_size = 0

        # Add the section to the current chunk
        current_chunk.append(section)
        current_size += section_size

    # Add the last chunk
    if current_chunk:
        chunks.append(''.join(current_chunk).strip())

    return chunks

# Chunk the text
chunks = robust_chunk_text_with_warning(content, max_chunk_size=500000)




Found 422 matches.


In [19]:
len(chunks[1])

401457